# Why xDSL?

Most programming languages are rather far removed from Assembly code. As a result, translating a language directly into Assembly is both difficult (easy to make mistakes) and inefficient (generated Assembly code is slow!). Therefore, most compilers goes through many intermediate representations / languages (IRs), which represents different layers of abstractions, to provide an efficient translation down to Assembly code - which is also feasible for compiler engineers to write. 

Although an effective approach, over the years it has become apparent that many compilers are repeatedly defining similar intermediate languages. Given the separate architecture of different compilers, it has been quite difficult to reuse code between architectures. 

Previous projects such as LLVM has provided a common abstraction layer for different compilers to reuse - however this is only one layer, and compilers of different domains are still having to repeat significant amounts of work to reach the LLVM level. 

LLVM is also not necessarily the ideal layer to perform optimisations on - thus many compilers will also implement their own implementations on custom IRs before they reach LLVM.

MLIR, and its Python parallel, xDSL, are frameworks to make it easy to:
1. Define custom abstraction layers for compilers
2. Reuse abstractions from other compilers
3. Encode domain-specific knowledge (i.e. optimisations) within these abstraction layers

<!-- Covered here: --!>
<!-- - How compilers use IRs --!>
<!-- - Why a framework for IRs are needed and is useful (ease of specifying optimisations)  --!>

# General Concepts

The kind of IRs that xDSL builds are SSA-based - a restriction placed on IRs which makes it easy for analysis of the code.

### Static Single Assignment (SSA) 
The kind of programming languages xDSL defines have the $\textit{static single assignment}$ (SSA) property. It is a property on variables that means they can be assigned a value only once, and once assigned a value, they cannot be modified. This is a common restriction used within compilers, as it enables many compiler optimisations to be performed.

In an SSA-based language, values are generated by the language's constructs, and old values may be referred to in the construction of new values.

<!-- give an example of SSA code snippet -->

Further reading: https://en.wikipedia.org/wiki/Static_single-assignment_form

# xDSL/MLIR
xDSL and MLIR are distinct frameworks for compiler IR specification, however, they are built upon similar basic concepts.

## What is an IR, anyway?

xDSL is a framework for modelling/specifying IRs. Before we go into the details of xDSL, it is perhaps useful to take a step back and consider what exactly an IR is!

Code can be represented in a human-readable textual format - however strings are difficult for compilers to handle. An IR is an *equivalent*, structured representation of code convenient for machines to parse through. Importantly, it is solely a representation of the code - and is not executable!

Languages are often designed for particular tasks and domains, which allows certain optimisations or types of reasoning to take place more easily within the language. A compiler might want to optimise the code in many different ways, and therefore can utilise multiple languages (represented as IR within the compiler) during its translation into machine code.

Wikipedia has a nice article as a starting point for further reading: https://en.wikipedia.org/wiki/Intermediate_representation

## Programming language syntax
All programming languages will have a set of "language construct", or, the _vocabulary_ available within the programming language.

These constructs can be composed together, forming "expressions" - which are sentences that _could_ be evaluated to a value. 

Just like in English, though, vocabulary (words) cannot be composed together arbitrarily - they should follow the syntactical rules of the language. For programming languages this is known as the _syntax_ of the language, and is often specified in terms of a [grammar](https://en.wikipedia.org/wiki/Formal_grammar).

Here are some examples of building blocks within various languages:
* Arithmetic: a language that describes arithmetic might have constructs including:
    * `+, -, x, /, round, max`
    * The values of this language are real numbers (floats, say). 
    * One syntax rule in this language is that each construct should take in either values, or other valid expressions.
* Python: includes too many constructs to fit in this code block, but some examples include: 
    * `for, if, while, assert, def, class, import`
    * Since Python is object-oriented, values in Python language are objects.
    * A valid expression in Python would follow the [Python grammar](https://docs.python.org/3/reference/grammar.html), a rather large document specifying all syntax rules of Python. 

## Programming language semantics
In English, a sentence being grammartically correct is not enough for it to have _meaning_. The meaning of sentences is embedded in the intuition of English speakers, and speakers need to generally agree on the meaning (semantics) of sentences in order to communicate.

Similarly for programming languages, the grammar is not enough to convey meaning. There needs to be some semantics of the language which is generally agreed across the users of the programming language and the compiler implementations. 

Programmers makes use of the language semantics to write programs which do what they think it does, and compilers utilise the semantics to optimise the programs whilst preserving its semantics - after all, the transformation should not change the meaning of what the programmer wrote! 

For programming languages, the semantics could be assigned to a mathematical function, unambiguously defining the meaning for all sentences in the programming languages.

However, in industrial programming languages, the semantics is often an intuitive notion that is embedded in the minds of language users and within the compiler's transformations (which is preserved).

## Programming language

With the above two definitions, a programming language's definition simply consists of two parts:
1. the syntax
2. the semantics

## xDSL: A tool for designing and implementing languages for compilers

Following from the previous definition of a programming language, the corresponding parts of a compiler IR consists of:
- The representation of the IR (i.e. the syntax - what constructs there are, how do they compose, what is the data structure to use in the implementation?)
- The transformations performed on the IR (i.e. the implied semantics: utilising the expressive powers of the IR in its domain for optimisations) 

xDSL provides the tools for modelling IRs, where:
- Modelling the IR's syntax is done via defining the _operations_, and _attributes_ within the dialect
- Modelling the IR's transformation is done via specifying _rewrite patterns_ on the IR

To be able to specify the syntax and the transformations of any IR, it needs to be general enough to fit the wide range of potential designs and uses of language constructs.

Below we will introduce the concepts used within xDSL.

### Operations: Modelling general language constructs

Operations are units used within MLIR and xDSL to model a "language construct", i.e. a vocabulary available within the language. 

A set of operations together would then describe the syntax of the language. 

An operation is the combination of: <!-- make it possible to click on the links to go to respective blocks perhaps -->
1. A name: the name of the operation
2. A (possibly empty) list of _regions_
3. A (possibly empty) list of _operands_
4. A (possibly empty) set of _attributes_
5. A (nonempty) list of results

Operations would take in some previously defined SSA values in the program (as operands), and return as a result new SSA value(s).

The above definition of an operation may seem rather terse - this is because an operation is designed to be general enough to describe all kinds of constructs within a prorgamming language. 

And so we will provide examples of operations below for more intuition. Moreover, please refer to the individual definitions for more information.

### Regions
A _region_ represents a sequence of instructions with potentially non-linear control flow.

That is, within a region, the program could execute not just from top-to-bottom, but it may go into different branches or loops. A region therefore corresponds to a control flow graph (_CFG_) - where the nodes are blocks, and edges are the flow of control: we have an edge when a block passes control to another block.

In MLIR/xDSL, a _region_ consists of:
1. A list of _blocks_ 

### Blocks
As commonly seen in compilers theory, blocks represent a sequence of instructions which must execute from top-to-bottom, one after the other. 

At the end of the block, the control flow is handed over to one of its potential _successors_ (which one exactly is dependent on the precise values computed, which is done at runtime)

What's different about blocks in xDSL/MLIR is that blocks contains operations, which themselves could contain regions, and regions can contain blocks once again. 

Thus, the structure of the IR is recursive within xDSL/MLIR.

In MLIR/xDSL, a block is the combination of:
1. A list of _block arguments_
2. A list of _operations_
3. A (possibly empty) list of _successor_ _blocks_

### Block arguments

Instead of using [$\phi$ nodes](https://www.cs.princeton.edu/~appel/papers/ssafun.pdf), xDSL/MLIR uses _block arguments_ to decide the value to use for computation when input values could come from several different branches.

These are two equivalent ways to solve the same problem, though, as explained in the earlier linked [article](https://www.cs.princeton.edu/~appel/papers/ssafun.pdf) on $\phi$ nodes.

In MLIR/xDSL, block arguments is the combination of:
1. An index, denoting the argument's position within the block
2. A type, i.e. the type of the argument

### Operands

Operands are the arguments to an operation. This allows the operation to use previously calculated values, which is essential for imperative programs.

Note that the operands are different to block arguments in that 
- _operands_ within an operation refers to a unique previously defined SSA values, whereas 
- _block arguments_ are more akin to function arguments, and do not specify a unique previously defined SSA value to use

In MLIR/xDSL, operands are defined as:
1. A SSA value: that is, the previously defined SSA value used in this operand

### Attributes

Attributes are data, in a predetermined format, which are associated with operations. Attribute's formats are specified by the language specification. 

The data it stores must be information which is known at compile time.

There are several uses of attributes with xDSL/MLIR, some examples include:
- **_type information_**: in xDSL, the types of operation's results are stored as attributes
- storage for code analysis: [dataflow analysis](https://clang.llvm.org/docs/DataFlowAnalysisIntro.html) passes computes properties such as liveness and range of variables, which relies on being able to store and update information within the IR
- operations whos semantics is dependent on attributes: such as 
    - [convolution operations](https://mlir.llvm.org/docs/Dialects/TOSA/#tosaconv2d-mlirtosaconv2dop) (semantics depends on its stride and padding attributes), 
    - [comparison operations](https://mlir.llvm.org/docs/Dialects/ArithOps/#arithcmpi-mlirarithcmpiop) (semantics depends on its "predicate" attribute, which specifies exactly what comparison to make, i.e. signed less than, unsigned greater than or equals, and so on)

### Dialects
Languages in xDSL are known as `dialects` - which is a grouping of related operations and attributes. For instance, 
- The `scf` (structured control flow) dialect groups together 
    - Operations such as: for/while loops, reduce, yield, if/else statements
    - Attributes such as: no new attributes in this dialect! It instead uses attributes/types from _other_ dialects
- The `arith` (arithmetic) dialect groups together
    - Operation such as: comparison, int/floating point addition/multiplication, or/xor/and logical bitwise operations, and so on
    - Attributes such as: predicates to decide the mode of comparison, flags to determine mode of floating point computation used
- The `async` (asynchornous) dialect groups together operations commonly seen in async applications, as well as associated attributes (types).
    - Operations such as: asynchronous calls, waits, async functions
    - Attributes such as: coroutine reference types, future types

Dialects can be _combined together_, such that the combined dialect have the operations and attributes from all constituent dialects.

### Rewrite Patterns
Rewrite pattern is a tool to transform IRs defined using xDSL/MLIR. 

It makes it easy to specify peephole optimisations. Rewrite patterns specifies a _source_ pattern, and a transformation, which applies itself to the IR by:
- matching a _source_ type pattern within the IR
- updating the IR by applying the predefined transformation

Multiple rewrite patterns can be applied and composed together in compiler passes.

A function signature is worth a thousand words...

```python
class RewritePattern(ABC):
    """
    A side-effect free rewrite pattern matching on a DAG.
    """

    @abstractmethod
    def match_and_rewrite(self, op: Operation, rewriter: PatternRewriter):
        """
        Match an operation, and optionally perform a rewrite using the rewriter.
        """
        # here, determine if the operation matches our pattern
        # and define a transformation to be applied on the IR making use of the 
        # `rewrite` as an API for changing the IR.
        ...
```

# xDSL Examples

## 1. A SQL dialect

As a first example, we'll describe a language for SQLs queries. This will be a simple language which is able to do two things: 
* reading from tables
* filtering results

### Grammar

### xDSL modelling

### Sample code in our SQL language

# DRAFT SECTION

# MLIR/xDSL examples
We want to demonstrate a range of MLIR/xDSL features

- An SQL-like representation
- A subset of Python, just for demonstration purposes
- Toy dialect
- Existing dialects, e.g. scf and arith

Would also be nice to have some examples of dialects being used together, e.g. the standard arith + if statements stuff?
- a range of dialect definitions with explanations, showcasing the usage/non-usage of regions, blocks, and attributes
- a range of MLIR/xDSL examples which makes use of the operations

# General format of examples
- Defining the language's grammar 
- Show how the language is modelled in xDSL
- Show some code snippets of the language, and how that translates to xDSL/MLIR SSA form
- Show some transformations run on the language. Lowering, and optimisations 